In [1]:
"initialize model and preprocessor. attn_implementation==eager"
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
from PIL import Image
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor

torch.set_grad_enabled(False)
model_dir = "/models/Qwen/Qwen2.5-VL-3B-Instruct"
model:Qwen2_5_VLForConditionalGeneration = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager",
).eval()
# model=torch.compile(model)
processor = Qwen2_5_VLProcessor.from_pretrained(model_dir)

/models/conda_envs/delius/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]


In [4]:
"tokenizer, preprocessor and inputs"
# Image
# url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
image = Image.open("../jinzhe_1024.jpg")
print(f"[Image Width] {image.width} [Image Height] {image.height}")
conversation = [
    {"role":"system","content":"你是一名高级人工智能助手。"},
    {
        "role": "user",
        "content": [
            {"type": "image","image":image},
            {"type": "text", "text": "描述下图片这个女孩"},
        ],
    }
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
print("[text_prompt]\n",text_prompt)
inputs = processor(text=[text_prompt], images=[image],padding=True,return_tensors='pt')
inputs = inputs.to("cuda")
print("[image_grid_thw] ",inputs.image_grid_thw)
print("[pixel_values shape] ",inputs.pixel_values.shape)
print("[input_ids[0,:-15] ] ",inputs.input_ids[0,-15:])
print("[input_ids shape] ",inputs.input_ids.shape)
input_ids=inputs.input_ids
pixel_values=inputs.pixel_values
image_grid_thw=inputs.image_grid_thw
# torch.save(pixel_values,"pixel_values.pt")
print(input_ids.tolist())

[Image Width] 1024 [Image Height] 1024
[text_prompt]
 <|im_start|>system
你是一名高级人工智能助手。<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>描述下图片这个女孩<|im_end|>
<|im_start|>assistant

[image_grid_thw]  tensor([[ 1, 74, 74]], device='cuda:0')
[pixel_values shape]  torch.Size([5476, 1176])
[input_ids[0,:-15] ]  tensor([151655, 151655, 151655, 151655, 151653,  53481,  16872,  45930,  99487,
        101339, 151645,    198, 151644,  77091,    198], device='cuda:0')
[input_ids shape]  torch.Size([1, 1395])
[[151644, 8948, 198, 56568, 110124, 104112, 104455, 110498, 1773, 151645, 198, 151644, 872, 198, 151652, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 15165

In [ ]:
"prefill stage, output image_embeds, inputs_embeds and logits"
inputs_embeds = model.model.embed_tokens(input_ids)
# torch.save(inputs_embeds, "inputs_embeds(before_embed_img).pt")

image_embeds = model.visual(pixel_values, grid_thw=image_grid_thw)
# torch.save(image_embeds, "image_embeds.pt")

mask = input_ids == model.config.image_token_id
mask_unsqueezed = mask.unsqueeze(-1)
mask_expanded = mask_unsqueezed.expand_as(inputs_embeds)
image_mask = mask_expanded.to(inputs_embeds.device)

image_embeds = image_embeds.to(inputs_embeds.device, inputs_embeds.dtype)
inputs_embeds = inputs_embeds.masked_scatter(image_mask, image_embeds)
# torch.save(inputs_embeds,"inputs_embeds(after_embed_img).pt")

In [ ]:
from transformers.generation.configuration_utils import GenerationConfig
generation_config:GenerationConfig=GenerationConfig(do_sample=False,top_p=1,temperature=0)
output_ids = model.generate(**inputs)[0,len(input_ids[0,:]):]
print(output_ids)
output_text = processor.decode(output_ids)
print(output_text)

/models/conda_envs/delius/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:626: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tensor([ 43288,  99639,  86341, 101919,  99329,  26940,  52129,  99315,  25067,
          9370, 101255,  54623,   3837, 109655,  99593,  24156,  57750, 100780,
         18493, 101272], device='cuda:0')
这是一张来自游戏《原神》的插画，描绘了一位女角色在战斗
